In [1]:
#!usr/bin/env python3
# -*- coding: utf-8 -*-

In [15]:
import tensorflow as tf
import numpy as np
import const as C
import util
import os

In [3]:
# filter = [kernel_height, kernel_width, input_channel, output_channel]
# 畳み込み層(エンコーダ部分)
def conv2d(x, output_channel, kernel=3, stride=2, batch_norm=True, is_training=True, leaky_relu=True):
    net = tf.layers.conv2d(x,
                          filters=output_channel,
                          kernel_size=[kernel, kernel],
                          strides=[stride, stride],
                          padding='SAME')
    if batch_norm:
        net = tf.layers.batch_normalization(net, training=is_training)
    if leaky_relu:
        net = tf.nn.leaky_relu(net, 0.2)
    return net

In [4]:
# filter = [kernel_height, kernel_width, output_channel, input_channel]
# output_shape = [バッチ数, 得たいheight, 得たいwidth, 得たいchannel]
# 逆畳み込み層(デコーダ部分)
def de_conv2d(x, output_channel, kernel=3, stride=2, batch_norm=True, is_training=True, relu=True):
    net = tf.layers.conv2d_transpose(x,
                                     filters=output_channel,
                                     kernel_size=[kernel, kernel],
                                     strides=[stride, stride],
                                     padding='SAME')
    if batch_norm:
        net = tf.layers.batch_normalization(net, training=is_training)
    if relu:
        net = tf.nn.relu(net)
    return net

In [5]:
def UNet():
    net = {}
    net['input'] = tf.Variable(np.zeros((1, C.IMAGE_HEIGHT, C.IMAGE_WIDTH, 1)).astype('float32'))
    net['conv1'] = conv2d(net['input'], output_channel=16)
    net['conv2'] = conv2d(net['conv1'], output_channel=32)
    net['conv3'] = conv2d(net['conv2'], output_channel=64)
    net['conv4'] = conv2d(net['conv3'], output_channel=128)
    net['conv5'] = conv2d(net['conv4'], output_channel=256)
    net['conv6'] = conv2d(net['conv5'], output_channel=512)
    net['de_conv1'] = de_conv2d(net['conv6'], output_channel=256)
    net['concat1'] = tf.concat([net['de_conv1'], net['conv5']], axis=-1)
    net['dropout1'] = tf.nn.dropout(net['concat1'], rate=0.5)
    net['de_conv2'] = de_conv2d(net['dropout1'], output_channel=128)
    net['concat2'] = tf.concat([net['de_conv2'], net['conv4']], axis=-1)
    net['dropout2'] = tf.nn.dropout(net['concat2'], rate=0.5)
    net['de_conv3'] = de_conv2d(net['dropout2'], output_channel=64)
    net['concat3'] = tf.concat([net['de_conv3'], net['conv3']], axis=-1)
    net['dropout3'] = tf.nn.dropout(net['concat3'], rate=0.5)
    net['de_conv4'] = de_conv2d(net['dropout3'], output_channel=32)
    net['concat4'] = tf.concat([net['de_conv4'], net['conv2']], axis=-1)
    net['de_conv5'] = de_conv2d(net['concat4'], output_channel=16)
    net['concat5'] = tf.concat([net['de_conv5'], net['conv1']], axis=-1)
    net['de_conv6'] = de_conv2d(net['concat5'], output_channel=1)
    net['activation_final'] = tf.math.sigmoid(net['de_conv6'])
    return net

In [6]:
model = UNet()

# 初期化
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.


In [7]:
X_list, y_list = util.load_dataset(target="vocal")
item_length = [x.shape[1] for x in X_list]

[14767,
 36074,
 44315,
 17906,
 22335,
 12657,
 14785,
 8583,
 24003,
 14940,
 13220,
 16860,
 21937,
 15111,
 26031,
 19254,
 25409,
 22191,
 17338,
 17623,
 19209,
 15664,
 20808,
 24412,
 18837,
 14642,
 24541,
 54140,
 9821,
 25668,
 24351,
 35549,
 15000,
 19737,
 3181,
 3133,
 3169,
 2993,
 1504,
 10748,
 6524,
 3606,
 1710,
 2478,
 1505,
 1128,
 2236,
 18346,
 27306,
 22070,
 14979,
 22640,
 30621,
 23704,
 24933,
 21005,
 33763,
 19652,
 26154,
 14385,
 16266]

In [11]:
print(len(item_length))
print(item_length)

61
[14767, 36074, 44315, 17906, 22335, 12657, 14785, 8583, 24003, 14940, 13220, 16860, 21937, 15111, 26031, 19254, 25409, 22191, 17338, 17623, 19209, 15664, 20808, 24412, 18837, 14642, 24541, 54140, 9821, 25668, 24351, 35549, 15000, 19737, 3181, 3133, 3169, 2993, 1504, 10748, 6524, 3606, 1710, 2478, 1505, 1128, 2236, 18346, 27306, 22070, 14979, 22640, 30621, 23704, 24933, 21005, 33763, 19652, 26154, 14385, 16266]


In [14]:
subepoch = sum(item_length) // C.PATCH_LENGTH // C.BATCH_SIZE * 4
subepoch

528

In [ ]:
class UNet(object):
    def __init__(self, batchsize=C.BATCH_SIZE, epoch=40, learning_rate=1e-4,
                 dropout_rate=0.5, shuffle=True, random_seed=None):
        np.random.seed(random_seed)
        self.batchsize = batchsize
        self.epoch = epoch
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        self.shuffle = shuffle
        
        g = tf.Graph()
        with g.as_default():
            # 乱数シードを設定
            tf.set_random_seed(random_seed)
            # モデルを構築
            self.build()
            # 変数を初期化
            self.init_op = tf.global_variables_initializer()
            # saver
            self.saver = tf.train.Saver()
        
        # セッションを作成
        self.sess = tf.Session(graph=g)
    
    def build(self):
        net = {}
        net['input'] = tf.Variable(np.zeros((1, C.IMAGE_HEIGHT, C.IMAGE_WIDTH, 1)).astype('float32'))
        net['conv1'] = conv2d(net['input'], output_channel=16)
        net['conv2'] = conv2d(net['conv1'], output_channel=32)
        net['conv3'] = conv2d(net['conv2'], output_channel=64)
        net['conv4'] = conv2d(net['conv3'], output_channel=128)
        net['conv5'] = conv2d(net['conv4'], output_channel=256)
        net['conv6'] = conv2d(net['conv5'], output_channel=512)
        net['de_conv1'] = de_conv2d(net['conv6'], output_channel=256)
        net['concat1'] = tf.concat([net['de_conv1'], net['conv5']], axis=-1)
        net['dropout1'] = tf.nn.dropout(net['concat1'], rate=0.5)
        net['de_conv2'] = de_conv2d(net['dropout1'], output_channel=128)
        net['concat2'] = tf.concat([net['de_conv2'], net['conv4']], axis=-1)
        net['dropout2'] = tf.nn.dropout(net['concat2'], rate=0.5)
        net['de_conv3'] = de_conv2d(net['dropout2'], output_channel=64)
        net['concat3'] = tf.concat([net['de_conv3'], net['conv3']], axis=-1)
        net['dropout3'] = tf.nn.dropout(net['concat3'], rate=0.5)
        net['de_conv4'] = de_conv2d(net['dropout3'], output_channel=32)
        net['concat4'] = tf.concat([net['de_conv4'], net['conv2']], axis=-1)
        net['de_conv5'] = de_conv2d(net['concat4'], output_channel=16)
        net['concat5'] = tf.concat([net['de_conv5'], net['conv1']], axis=-1)
        net['de_conv6'] = de_conv2d(net['concat5'], output_channel=1)
        net['activation_final'] = tf.math.sigmoid(net['de_conv6'])
        
        # 損失関数
        loss = tf.keras.metrics.mean_absolute_error(net['input']*net['activation_last'], y)
        # 最適化
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        optimizer = optimizer.minimize(loss. name='train_op')
        
        def save(self, epoch, path='./tf-layers-model/'):
            if not os.path.isdir(path):
                os.makedirs(path)
            
            print('Saving model in %s' %path)
            self.saver.save(self.sess,
                            os.path.join(path, 'model.ckpt'),
                            global_step=epoch)
        
        def load(self, epoch, path):
            print('Loading model from %s' %path)
            self.saver.restore(self.sess,
                               os.path.join(path, 'model.ckpt-%d' % epoch))